## Implementation of RFUAV-net
July 1, 2022 \
efficient CNN method \
status: tested with a small sample, did not run the full model

In [22]:
import os
import numpy as np
from sklearn.model_selection import KFold
import pandas as pd
from sklearn import preprocessing

In [2]:
## helper functions
def normalize_rf(rf):
    """apply normalization to data in the numpy array format"""
    for i in range(len(rf)):
        for j in range(2):
            rf[i][j] = (rf[i][j] - np.min(rf[i][j]))/(np.max(rf[i][j])-np.min(rf[i][j]))

    return rf

In [3]:
## Import data -  Drone RF
main_folder = '/home/kzhou/Data/DroneRF/'
high_freq_files = os.listdir(main_folder+'High/')
low_freq_files = os.listdir(main_folder+'Low/')

high_freq_files.sort()
low_freq_files.sort()
fs = 40e6 #40 MHz

# feature & results lists
Xs = []
ys = []
y4s = []
y10s = []

for i in range(100):

    # load RF data
    rf_data_h = pd.read_csv(main_folder+'High/'+high_freq_files[i], header=None).values
    rf_data_h = rf_data_h.flatten()
    
    rf_data_l = pd.read_csv(main_folder+'Low/'+low_freq_files[i], header=None).values
    rf_data_l = rf_data_l.flatten()
    
    if len(rf_data_h)!=len(rf_data_l):
        print('diff', i, 'file name:', low_freq_files[i]) 
        # not sure why one pair of files have different lengths (ignore this for now)
    else:
        # stack the features and ys
        rf_sig = np.vstack((rf_data_h, rf_data_l))
        rf_sig = np.split(rf_sig, 1000, axis =1) # samples of 1e4
        Xs.append(normalize_rf(rf_sig))
        
        y_rep = np.repeat(int(low_freq_files[i][0]),1000)
        y4_rep = np.repeat(int(low_freq_files[i][:3]),1000)
        y10_rep = np.repeat(int(low_freq_files[i][:5]),1000)

        ys.append(y_rep) # 2 class
        y4s.append(y4_rep) # 4 class
        y10s.append(y10_rep) # 10 class

        if int(high_freq_files[i][:5])!= int(low_freq_files[i][:5]):
            raise Exception("File labels do not match")

diff 5 file name: 00000L_13.csv


In [16]:
# shape the arrays
Xs_arr = np.array(Xs)
Xs_arr = Xs_arr.reshape(-1, *Xs_arr.shape[-2:])
ys_arr = np.array(ys).flatten()
y4s_arr = np.array(y4s).flatten()
y10s_arr = np.array(y10s).flatten()

In [17]:
Xs_arr.shape

(99000, 2, 10000)

### Apply Model

In [18]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch
import torchvision.models as models

In [19]:
## Define dataset
## Create a dataset class
## Creating a custom dataset
class DroneRFData(Dataset): ## NUMBERICAL DATA
    def __init__(self, Xarr, yarr):
        self.Xarr = Xarr
        test_list=[]
        le = preprocessing.LabelEncoder()
        le.fit(yarr.flatten())
        self.yarr = le.transform(yarr.flatten())
        
    def __len__(self):
        return len(self.yarr)
    
    def __getitem__(self, index):
        # all data must be in float and tensor format
        X = torch.tensor((self.Xarr[index]))
        X = X.unsqueeze(0)
        y = torch.tensor(float(self.yarr[index]))
        return (X, y)

In [23]:
dataset = DroneRFData(Xs_arr, ys_arr)

In [31]:
dataset.__getitem__(10)[0][0][0].min()

tensor(0)

In [ ]:
class RFUAVNet(nn.Module):
    #  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.avpool0 = nn.AvgPool2d(kernel_size=(50,200))
        self.dense = nn.Linear(1670, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
         # Max pooling over a (2, 2) window
        x = self.avpool0(x)
        x = torch.flatten(x,1)
        x = self.dense(x)
        return x
    
    def runit(self,x):
        